In [1]:
pip install pandas scikit-learn numpy pillow keras-tuner


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, LSTM, Dense, Dropout, BatchNormalization, Reshape
from keras.applications import DenseNet121
from keras.regularizers import l2
from keras_tuner import BayesianOptimization, HyperParameters
from keras.callbacks import EarlyStopping
from keras_tuner.engine.hypermodel import HyperModel

# Define the root directory where your image folders are located
root_directory = "PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path']).convert('RGB')
        image_array = np.array(image.resize((256, 256)))  # Resize image to fit DenseNet input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)


2024-12-21 20:01:41.279230: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 256, 256, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 256, 256, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 256, 256, 3) (6909,)
Ischaemia Validation set shape: (2220, 256, 256, 3) (2220,)
Ischaemia Test set shape: (741, 256, 256, 3) (741,)
Infection Training set shape: (4123, 256, 256, 3) (4123,)
Infection Validation set shape: (1325, 256, 256, 3) (1325,)
Infection Test set shape: (442, 256, 256, 3) (442,)


In [5]:
pip install py-cpuinfo



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
class MyHyperModel(HyperModel):

    def __init__(self):
        self.base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
        for layer in self.base_model.layers:
            layer.trainable = False

    def build(self, hp):
        model = Sequential([
            self.base_model,
            GlobalAveragePooling2D(),
            Reshape((8, 128)),
            LSTM(
                units=hp.Int('units_lstm1', min_value=16, max_value=32, step=8),
                return_sequences=True,
                kernel_regularizer=l2(hp.Float('l2_lstm1', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_lstm1', min_value=0.4, max_value=0.5, step=0.1)),
            LSTM(
                units=hp.Int('units_lstm2', min_value=16, max_value=32, step=8),
                return_sequences=True,
                kernel_regularizer=l2(hp.Float('l2_lstm2', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_lstm2', min_value=0.4, max_value=0.5, step=0.1)),
            LSTM(
                units=hp.Int('units_lstm3', min_value=16, max_value=32, step=8),
                kernel_regularizer=l2(hp.Float('l2_lstm3', min_value=0.001, max_value=0.01, step=0.001))
            ),
            BatchNormalization(),
            Dense(
                units=hp.Int('units_dense1', min_value=32, max_value=64, step=16),
                activation='relu',
                kernel_regularizer=l2(hp.Float('l2_dense1', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_dense1', min_value=0.4, max_value=0.5, step=0.1)),
            BatchNormalization(),
            Dense(
                units=hp.Int('units_dense2', min_value=32, max_value=64, step=16),
                activation='relu',
                kernel_regularizer=l2(hp.Float('l2_dense2', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_dense2', min_value=0.4, max_value=0.5, step=0.1)),
            BatchNormalization(),
            Dense(3, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

# Instantiate the HyperModel
hypermodel = MyHyperModel()


2024-12-21 20:07:13.366719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22066 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:42:00.0, compute capability: 8.9
2024-12-21 20:07:13.369230: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22066 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:61:00.0, compute capability: 8.9


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step 


In [6]:
import cpuinfo

info = cpuinfo.get_cpu_info()
features = info.get('flags', [])

if 'avx2' in features:
    print("AVX2 is supported.")
else:
    print("AVX2 is not supported.")

if 'fma' in features:
    print("FMA is supported.")
else:
    print("FMA is not supported.")


AVX2 is supported.
FMA is supported.


In [8]:
tuner = BayesianOptimization(
    hypermodel,
    objective='val_accuracy',
    max_trials=5,
    directory='infection_dir1',
    project_name='infection_model'
)

In [9]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tuner.search(X_train_infection, y_train_infection, epochs=50, validation_data=(X_val_infection, y_val_infection), callbacks=[reduce_lr, early_stopping])

Trial 5 Complete [00h 06m 18s]
val_accuracy: 0.7433962225914001

Best val_accuracy So Far: 0.7547169923782349
Total elapsed time: 00h 30m 26s


In [10]:
best_model_infection = tuner.get_best_models(num_models=1)[0]
test_loss_infection, test_accuracy_infection = best_model_infection.evaluate(X_test_infection, y_test_infection)
print(f"Infection Test Accuracy: {test_accuracy_infection}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 44 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.7594 - loss: 0.5367
Infection Test Accuracy: 0.7556561231613159


In [ ]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score

# Start time for training
start_training_time = time.time()

# Get the best hyperparameters and build the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Fit the model and collect the training metrics
history = best_model.fit(X_train_infection, y_train_infection, epochs=20, batch_size=32, 
                         validation_data=(X_val_infection, y_val_infection), 
                         callbacks=[early_stopping, reduce_lr])

# Collect training metrics
train_accuracy = history.history['accuracy'][-1]
train_loss = history.history['loss'][-1]
train_f1 = f1_score(y_train_infection, np.argmax(best_model.predict(X_train_infection), axis=1), average='macro')
train_precision = precision_score(y_train_infection, np.argmax(best_model.predict(X_train_infection), axis=1), average='macro')
train_recall = recall_score(y_train_infection, np.argmax(best_model.predict(X_train_infection), axis=1), average='macro')

# End time for training
end_training_time = time.time()
training_time = end_training_time - start_training_time

# Start time for validation
start_validation_time = time.time()

# Validate the model
val_loss, val_accuracy = best_model.evaluate(X_val_infection, y_val_infection)

# Collect validation metrics
val_f1 = f1_score(y_val_infection, np.argmax(best_model.predict(X_val_infection), axis=1), average='macro')
val_precision = precision_score(y_val_infection, np.argmax(best_model.predict(X_val_infection), axis=1), average='macro')
val_recall = recall_score(y_val_infection, np.argmax(best_model.predict(X_val_infection), axis=1), average='macro')

# End time for validation
end_validation_time = time.time()
validation_time = end_validation_time - start_validation_time

# Start time for testing
start_testing_time = time.time()

# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test_infection, y_test_infection)

# Predictions for classification metrics
y_pred = best_model.predict(X_test_infection)
y_pred_classes = np.argmax(y_pred, axis=1)

# Collect testing metrics
test_f1 = f1_score(y_test_infection, y_pred_classes, average='macro')
test_precision = precision_score(y_test_infection, y_pred_classes, average='macro')
test_recall = recall_score(y_test_infection, y_pred_classes, average='macro')

# End time for testing
end_testing_time = time.time()
testing_time = end_testing_time - start_testing_time

# Collect all metrics in a dictionary for easy access
metrics = {
    'Training Accuracy': train_accuracy,
    'Training Loss': train_loss,
    'Training F1 Score': train_f1,
    'Training Precision': train_precision,
    'Training Recall': train_recall,
    'Training Time (s)': training_time,
    'Validation Accuracy': val_accuracy,
    'Validation Loss': val_loss,
    'Validation F1 Score': val_f1,
    'Validation Precision': val_precision,
    'Validation Recall': val_recall,
    'Validation Time (s)': validation_time,
    'Test Accuracy': test_accuracy,
    'Test F1 Score': test_f1,
    'Test Precision': test_precision,
    'Test Recall': test_recall,
    'Testing Time (s)': testing_time
}

# Print the collected metrics
for metric, value in metrics.items():
    print(f"{metric}: {value}")

Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 0.3874 - loss: 2.2677 - val_accuracy: 0.4853 - val_loss: 1.6892 - learning_rate: 0.0010
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.4992 - loss: 1.7688 - val_accuracy: 0.5675 - val_loss: 1.4239 - learning_rate: 0.0010
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.5808 - loss: 1.4495 - val_accuracy: 0.5909 - val_loss: 1.2796 - learning_rate: 0.0010
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.6041 - loss: 1.2837 - val_accuracy: 0.6506 - val_loss: 1.1497 - learning_rate: 0.0010
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.6440 - loss: 1.1493 - val_accuracy: 0.6740 - val_loss: 1.0604 - learning_rate: 0.0010
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.6605 - loss: 1.0596 - val_accuracy: 0.6845 - val_loss: 0.9907 - learning_rate: 0.0010
Epoch 7/20
 11/129 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.6867 - loss: 0